In [1]:
import merlin.models.tf as mm
from merlin.schema.tags import Tags
import tensorflow as tf
from merlin.models.tf.blocks import *
from merlin.datasets.synthetic import generate_data
import pandas as pd

import numpy as np

2023-05-19 07:06:25.875214: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")


2023-05-19 07:06:28.324492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-19 07:06:28.324884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-19 07:06:28.325029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-05-19 07:06:29.896353: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 07:06:29.897503: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-19 07:06:29.897741: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-19 07:06:29.897922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must 

In [2]:
import cudf

In [3]:
ls /workspace/sigir_dataset/train

browsing_train.csv      search_train.csv
browsing_train.parquet  sku_to_content.csv


If we attempt reading in the data, we unfortunately get a `cudf` error.

In [4]:
cudf.read_csv('/workspace/sigir_dataset/train/browsing_train.csv')

RuntimeError: cuDF failure at: /opt/rapids/src/cudf/cpp/include/cudf/strings/detail/strings_column_factories.cuh:88: total size of strings is too large for cudf column

`cudf` can only read in datasets that have fewer than 2**32 characters. Here there is so much data and the strings are so long we cross this limit.

We could work with only partial data. But we would still have the problem of our id column consisting of strings. This is something that `datasets` is unable to work with for synthetic data generation.

Let's instead "categorify" the string columns of interest. We will have to do this nonetheless if we will want to work with the full dataset in `cudf`.

In [52]:
train = pd.read_csv('/workspace/sigir_dataset/train/browsing_train.csv')

In [36]:
train.head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
2,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
3,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
4,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...


In [7]:
train.shape[0] / 1_000_000

36.079307

36+ million records!

In [37]:
len(train.iloc[0]['session_id_hash'])

64

In [38]:
2**32 / (train.shape[0] * 64)

1.860037500165954

We cannot accommodate even two hash columns!

In [53]:
train['session_id_hash'] = train['session_id_hash'].astype('category')
categories_index = train['session_id_hash'].cat.categories
train['session_id_hash'] = train['session_id_hash'].cat.codes

train['product_sku_hash'] = train['product_sku_hash'].astype('category')
categories_index = train['product_sku_hash'].cat.categories
hash_to_cat_idx = {h: i for i, h in enumerate(categories_index)}
train['product_sku_hash'] = train['product_sku_hash'].cat.codes

train['hashed_url'] = train['hashed_url'].astype('category')
categories_index = train['hashed_url'].cat.categories
train['hashed_url'] = train['hashed_url'].cat.codes

train.head()

This is looking good and we are able to output and read in the file using `cudf`.

In [48]:
train.to_parquet('/workspace/sigir_dataset/train/browsing_train.parquet')
cudf.read_parquet('/workspace/sigir_dataset/train/browsing_train.parquet').head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,631811,event_product,detail,47936,1550885210881,241663
1,631811,event_product,detail,22091,1550885213307,151100
2,631811,pageview,<NA>,-1,1550885213307,151100
3,631811,event_product,detail,47936,1550885215484,241663
4,631811,pageview,<NA>,-1,1550885215484,241663


For training on the full dataset, we would need to apply similar processing to the sku information (using the hash to index mapping we collected above).

In [49]:
ls /workspace/sigir_dataset/train

browsing_train.csv      search_train.csv
browsing_train.parquet  sku_to_content.csv


In [68]:
skus = pd.read_csv('/workspace/sigir_dataset/train/sku_to_content.csv')

skus['product_sku_hash'] = skus['product_sku_hash'].map(hash_to_cat_idx).fillna(0).astype('int')

This will allow us to link sessions data to pretrained embeddings.

Let's now generate synthetic data.

In [148]:
train = pd.read_csv('/workspace/sigir_dataset/train/browsing_train.csv').head(10_000)

train['session_id_hash'] = train['session_id_hash'].astype('category')
categories_index = train['session_id_hash'].cat.categories
train['session_id_hash'] = train['session_id_hash'].cat.codes

train['product_sku_hash'] = train['product_sku_hash'].astype('category')
categories_index = train['product_sku_hash'].cat.categories
hash_to_cat_idx = {h: i for i, h in enumerate(categories_index)}
train['product_sku_hash'] = train['product_sku_hash'].cat.codes

train['hashed_url'] = train['hashed_url'].astype('category')
categories_index = train['hashed_url'].cat.categories
train['hashed_url'] = train['hashed_url'].cat.codes

train.head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,194,event_product,detail,1257,1550885210881,1320
1,194,event_product,detail,593,1550885213307,832
2,194,pageview,NaN,-1,1550885213307,832
3,194,event_product,detail,1257,1550885215484,1320
4,194,pageview,NaN,-1,1550885215484,1320


In [198]:
train['event_type'] = train['event_type'].fillna('unknown')
train['product_action'] = train['product_action'].fillna('unknown')

In [199]:
train.session_id_hash.nunique()

1363

In [200]:
from nvtabular import *
import nvtabular as nvt

In [201]:
out = ['session_id_hash'] >> nvt.ops.Categorify(max_size=1000) >> nvt.ops.TagAsItemID()
out += ['event_type'] >> nvt.ops.Categorify(max_size=1000)

In [202]:
train.columns.tolist()

['session_id_hash',
 'event_type',
 'product_action',
 'product_sku_hash',
 'server_timestamp_epoch_ms',
 'hashed_url']

In [203]:
wf = Workflow(out + train.columns.tolist()[2:])

In [210]:
ds = Dataset(train)
ds_out = wf.fit_transform(ds)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [211]:
ds_out.to_parquet('ds_out')

In [212]:
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata

pb = TensorflowMetadata.from_proto_text_file(path='ds_out')
pb.to_json_file(path='ds_out')

!python3 -m json.tool ds_out/schema.json

{
    "feature": [
        {
            "name": "session_id_hash",
            "type": "INT",
            "intDomain": {
                "name": "session_id_hash",
                "max": "999",
                "isCategorical": true
            },
            "annotation": {
                "tag": [
                    "item_id",
                    "item",
                    "id",
                    "categorical"
                ],
                "extraMetadata": [
                    {
                        "num_buckets": null,
                        "freq_threshold": 0.0,
                        "max_size": 1000.0,
                        "start_index": 0.0,
                        "cat_path": ".//categories/unique.session_id_hash.parquet",
                        "embedding_sizes": {
                            "cardinality": 1000.0,
                            "dimension": 77.0
                        },
                        "_dims": [
                            [
      

In [213]:
from merlin.datasets.synthetic import generate_data

In [214]:
generate_data('ds_out/schema.json', 1000)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


ValueError: The registered dtype mapping for numpy doesn't contain type unknown.